In [1]:
import os
import sys
import time
import json

import urllib
import urllib.request
import requests

import pandas as pd
from bs4 import BeautifulSoup

import re

from tqdm import tqdm

In [5]:
# data_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieInfo\hot"
# save_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieReview"
data_path = r"C:\Users\chongs\Desktop\git\final_project\DataScience\Final_Data\movie\movieInfo\hot"
save_path = r"C:\Users\chongs\Desktop\git\final_project\DataScience\Final_Data\movie\movieReview"
# data_path = r"D:\git_ec\final_project\DataScience\Final_Data\movie\movieInfo\all"
# save_path = r"D:\git_ec\final_project\DataScience\Final_Data\movie\movieReview"

In [6]:
movies = pd.read_csv(os.path.join(data_path, "movies_hot_merged_en.csv"), encoding="euc-kr")

In [8]:
def progress_bar(i, total):
    k = i/total*100
    star = "*"*int(k/2.5)
    print("Processing: {}/{} [{:.1f}%] {}".format(i, total, k, star), end="\r")

In [9]:
def get_naverApi(movieNm, openDt, director):
    
    client_id = "JjcGyzzf5rjyFKNPCTxG"
    client_secret = "MDi_1o76AT"
    encText = urllib.parse.quote(f"{movieNm}")
    url = "https://openapi.naver.com/v1/search/movie?query=" + encText + f"&yearfrom={openDt-1}&yearto={openDt}"
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode == 200):
        html = response.read().decode("utf-8")
        parse = json.loads(html)
                
        if parse["items"] == []:
            link = ""
            
            
        else:
            if len(parse["items"]) > 1:
                for i, item in enumerate(parse["items"]):
                    if director in item["director"]:
                        idx = i
                    elif item["pubDate"] == str(openDt):
                        idx = i
                    else:
                        idx = 0
            else:
                 idx = 0
            
            info = parse["items"][idx]
            link = info["link"]
            
    else:
        link = ""
    
    return movieNm, link

In [10]:
def load_naverMovie(movie_list):
    movie_list.openDt = movie_list.openDt.astype(str)
    ls = movie_list[["movieCd","movieNm", "openDt", "directors"]].values
    
    i = 1
    total = len(ls)
    links = []
    movieNms = []
    movieCds = []
    for v in ls:
        movieNm, link = get_naverApi(v[1], int(v[2][:4]), v[3].split(" ")[0].split(",")[0])
        code = v[0]
        links.append(link)
        movieNms.append(movieNm)
        movieCds.append(code)
        progress_bar(i, total)
        i += 1
        time.sleep(0.1)

    print()
    print("complete!")
    
    return movieCds, movieNms, links

In [11]:
movieNm, link = get_naverApi("타이타닉", 1997, "제임스 카메론")

In [12]:
movieNm, link

('타이타닉', 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=18847')

In [121]:
naver_movie_code = link[51:]

In [48]:
target_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=179482'

In [67]:
res = requests.get(target_url)
soup = BeautifulSoup(urllib.request.urlopen(target_url).read(),'lxml')

C:\Users\chongs\Anaconda3\envs\mypython\lib\site-packages\beautifulsoup4-4.7.1-py3.6.egg\bs4\__init__.py:335: UserWarning: "https://movie.naver.com/movie/bi/mi/basic.nhn?code=179482" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [68]:
soup.find('svg')

In [52]:
scripts = soup.select('script[type="text/javascript"]')

In [60]:
script = [script.text for script in scripts if '%' in script.text ]

In [61]:
script

['\n\tvar gnb_service = "movie";\n\tvar gnb_logout = "http://movie.naver.com/movie/bi/mi/basic.nhn%3Fcode%3D179482";\n\tvar gnb_template = "gnb_utf8";\n\tvar gnb_brightness=3;\n\tvar gnb_response = true;\n\t\n\tjindo.$Fn(function(){\n\t\tgetGNB();\n\t}).attach(window, "load");\n\t\n\tjindo.$Fn(function(oEvent){ var welSource = jindo.$Element(oEvent.element);\n\t\t\tif (!welSource.isChildOf(jindo.$Element("gnb"))) {\n\t\t\t\tgnbAllLayerClose();\n\t\t\t}\n\t}).attach(document, \'click\');\n',
 '\r\n  \tvar oActualGenderPointChart =null;\r\n    oActualGenderPointChartWide = null\r\n\tvar today = new Date();\r\n\tvar dd = today.getDate();\r\n\tvar mm = today.getMonth() + 1;\r\n\tvar yyyy = today.getFullYear();\r\n\t\r\n\tif (dd < 10) {\r\n\t\tdd = \'0\' + dd;\r\n\t}\r\n\t\r\n\tif (mm < 10) {\r\n\t\tmm = \'0\' + mm;\r\n\t}\r\n\t\r\n\tvar d = document.head ? document.head : document\r\n\t\t\t.getElementsByTagName("head")[0];\r\n\tvar s = document.createElement("script");\r\n\ts.src = ("https

In [44]:
soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "html5lib")

In [45]:
gender_graph = soup.find("div",{ "id" : "actualGenderGraph" })

In [46]:
gender_graph

<div class="graph_box" id="actualGenderGraph" style="width: 100px;height:100px;">
						<!--[D] 그래프 영역 -->
					</div>

In [47]:
soup.select('.donut_graph')

[<div class="donut_graph">
                 <div class="graph_box" id="actualGenderGraph_wide" style="width: 100px;height:100px;">
 				</div>
 
 				<div class="graph_legend">
 					<ul>
 						<li>남자</li>
 						<li>여자</li>
 					</ul>
 				</div>
 			</div>, <div class="donut_graph">
 					<div class="graph_box" id="actualGenderGraph" style="width: 100px;height:100px;">
 						<!--[D] 그래프 영역 -->
 					</div>
 
 					<div class="graph_legend">
 						<ul>
 							<li>남자</li>
 							<li>여자</li>
 						</ul>
 					</div>
 				</div>]

In [31]:
gender_graph.find('text')

In [27]:
soup.select("div",{ "id" : "actualGenderGraph" })

[<div class="basic" id="wrap">
 <!-- GNB -->
 <script type="text/javascript">
 function delayed_submit(object) {
 	if (navigator.userAgent.indexOf('MSIE') == -1) {
 		var b = c = new Date();
       	while ((b.getTime() - c.getTime()) < 100) {
 			b = new Date();
       	}
 	} 
 }
 </script>
 <script type="text/javascript">
 	var gnb_service = "movie";
 	var gnb_logout = "http://movie.naver.com/movie/bi/mi/basic.nhn%3Fcode%3D179482";
 	var gnb_template = "gnb_utf8";
 	var gnb_brightness=3;
 	var gnb_response = true;
 	
 	jindo.$Fn(function(){
 		getGNB();
 	}).attach(window, "load");
 	
 	jindo.$Fn(function(oEvent){ var welSource = jindo.$Element(oEvent.element);
 			if (!welSource.isChildOf(jindo.$Element("gnb"))) {
 				gnbAllLayerClose();
 			}
 	}).attach(document, 'click');
 </script>
 <!-- skip navigation -->
 <div id="u_skip">
 <a href="#header" onclick="document.getElementById('header').tabIndex=-1;document.getElementById('header').focus();return false;"><span>메인 메뉴로 바로가기</span>

In [39]:
def get_reviews(naver_movie_code):
    target_url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={naver_movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
    soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "html.parser")
    review_total = soup.find("div",class_="score_total")
    
#     review가 없는 경우는 아무것도 리턴하지 않는다
    if review_total is not None:
        number = review_total.find('em').text
        number = int(re.sub(',', '', number))
        page_num = number //10

        user_id = []
        review = []
        score = []
        
        for page in range(1, page_num):
            target_url_page = target_url + '&page={}'.format(page)
            soup = BeautifulSoup(urllib.request.urlopen(target_url_page).read(), "html.parser")
            review_divs = soup.find_all('div',class_='score_reple')
            score_divs = soup.find_all('div', class_="star_score")
            for spans in review_divs:
                txt = spans.find('p').text.strip()
                if txt is not None:
                    review.append(txt)
                else:
                    review.append('')
            for spans in review_divs:
                dl = spans.find('dl')
                if dl is not None:
                    user = dl.find('span').text
                    user_id.append(user)
                else:
                    user_id.append('')
            for span in score_divs:
                if span is not None:
                    s = span.find('em').text.strip()
                    score.append(s)
                else:
                    score.append('')
            
        return user_id, review, score
    else:
        return ["", "", ""]

In [73]:
movieCds, movieNms, links = load_naverMovie(movies[5:10])

Processing: 5/5 [100.0%] ****************************************
complete!


In [75]:
movieNms

['어벤져스: 엔드게임', '알라딘', '7번방의 선물', '겨울왕국', '어바웃 타임']

In [ ]:
user_id = []
review = []
score = []
for link in tqdm(links):
    code = link[51:]
    ids, reply, star_score = get_reviews(code)
    user_id.append(ids)
    review.append(reply)
    score.append(star_score)


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

In [58]:
len(user_id[0]), len(review[0]), len(score[0])

(40940, 40940, 40940)

In [ ]:
for i in range(5):
    df = pd.DataFrame([user_id[i], review[i], score[i]], index=["user_id", "review", "score"]).T
    df.to_csv(os.path.join(save_path, f"{movieCds[i]} {movieNms[i]}.csv"), encoding="utf-8")

In [64]:
reviews = []
for i in range(5):
    reviews.append(pd.DataFrame([user_id[i], review[i], score[i]], index=["user_id", "review", "score"]).T)

In [72]:
for i in range(5):
    reviews[i].to_csv(os.path.join(save_path, f"{movies.iloc[i].movieCd} {movies.iloc[i].movieNm}.csv"), encoding="utf-8")